<a href="https://colab.research.google.com/github/Kieranosaurus/HYPER/blob/master/MXNet_PGGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet

import mxnet as mx
from mxnet import nd, gluon, init, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from IPython import display
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle

In [2]:
def load_weights():
    with open('pggan_weights', 'rb') as f:
        wb = pickle.load(f)
    return wb

In [138]:
class Net(gluon.Block):

    @staticmethod
    def pixel_norm(x, epsilon=1e-8):
        return x * nd.rsqrt(nd.mean(nd.square(x), axis=1, keepdims=True) + epsilon)

    @staticmethod
    def apply_bias(self, x):
        b = self.params.get('bias', shape=[x.shape[1]], init=init.Zero())
        b.initialize()
        b = b.data()
        print(b.shape)
        print(b)
        b = nd.zeros(shape=[x.shape[1]])
        b = nd.cast(b, x.dtype)
        if len(x.shape) == 2:
            return x + b
        else:
            b = b.reshape([1,-1,1,1])
            return x + b

    @staticmethod
    def upscale_2d(x, factor=2):
        s = x.shape()
        x = nd.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
        x = nd.tile(x, [1, 1, 1, factor, 1, factor])
        x = nd.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
        return x

    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            # toRGB
            self.torgb = nn.Conv2D(channels=3, kernel_size=1, strides=(1,1), layout='NCHW', use_bias=False)

            # Leaky ReLU Activation
            self.lrelu = nn.LeakyReLU(alpha=0.2)

            # Layer 1
            self.dense = nn.Dense(units=512, use_bias=False)
            self.conv1 = nn.Conv2D(channels=512, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 2
            self.conv2 = nn.Conv2D(channels=512, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 3
            self.conv3 = nn.Conv2D(channels=512, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 4
            self.conv4 = nn.Conv2D(channels=512, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 5
            self.conv5 = nn.Conv2D(channels=256, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 6
            self.conv6 = nn.Conv2D(channels=128, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 7
            self.conv7 = nn.Conv2D(channels=64, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 8
            self.conv8 = nn.Conv2D(channels=32, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

            # Layer 9
            self.conv9 = nn.Conv2D(channels=16, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

    def forward(self, x):
        # Layer 1
        x = self.pixel_norm(x)                                  #182
        x = self.dense(x)                                       #184
        x = x.reshape([-1, 512, 4, 4])                         #185
        x = self.pixel_norm(self.lrelu(self.apply_bias(self, x)))               #186
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv1(x))))   #188
        # torgb? -> for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 2
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv2(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv2(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 3
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv3(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv3(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 4
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv4(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv4(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 5
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv5(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv5(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 6
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv6(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv6(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 7
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv7(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv7(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 8
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv8(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv8(x))))   #198
        # torgb for intermediate output generation
        # y = self.apply_bias(self.torgb(x))                           #203

        # Layer 9
        x = self.upscale_2d(x)                                       #194
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv9(x))))   #196
        x = self.pixel_norm(self.lrelu(self.apply_bias(self.conv9(x))))   #198
        
        
        return x

In [ ]:
wb = load_weights()
print(len(wb),"\n")
for i in range(len(wb)):
    print("Index %d: %d" % (i, len(wb[i])))

In [139]:
net = Net()
net.collect_params().initialize(init.Normal())

In [6]:
# Get latents from test data stimuli
with open('data_sub1_4096.dat', 'rb') as fp:
    _, _, _, T_test = pickle.load(fp)

In [140]:
faces = []
latents = nd.array(T_test)
for i in range(latents.shape[0]):
    latent = nd.expand_dims(latents[i], 0)
    faces.append(net(latent))

(512,)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

MXNetError: ignored